# Timelag Analysis: Nonequilibrium versus Equilibrium Ionization
For one of our heating cases (likely intermediate-frequency heating), compare the timelag distributions for intensities computed accounting for non-equilibrium ionization and assuming equilibrium ionization

In [ ]:
import os
import sys

import numpy as np
import dask
import distributed
import matplotlib.pyplot as plt
import matplotlib.colors
from sunpy.map import Map,GenericMap
from astropy.coordinates import SkyCoord
import astropy.units as u

import synthesizAR
from synthesizAR.instruments import InstrumentSDOAIA
from synthesizAR.visualize import plot_aia_channels

sys.path.append('../scripts/')
from timelags import AIATimeLags

%matplotlib inline